<!--- mission_number=6 file_list=["hn_stories.csv", "stop_words.txt"] mode="singlescreen" --->

# Predicting HN upvotes using headlines
## Use natural language processing and linear regression to predict upvotes from headlines.
## Vik Paruchuri

<!--- type="markdown" --->

# Looking at the data

We'll be looking at a dataset consisting of submissions to [hacker news](http://news.ycombinator.com) from 2006 to 2015.  The data was taken from [here](https://github.com/arnauddri/hn) where Arnaud Drizard used the Hacker News API to scrape it.  We've sampled `100000` rows from the data randomly, and removed all the extraneous columns.  We only have four columns:

* `submission_time` -- when the story was submitted.
* `url` -- the base url of the submission.
* `upvotes` -- number of upvotes the submission got.
* `headline` -- the headline of the submission.

We'll be using the headline to predict number of upvotes.  The data is stored in the `submissions` variable.

<!--- type="code" --->

# First steps

We want to eventually train an algorithm that can be passed in a headline, and tell us how many upvotes it would get.  However, these algorithms only understand numbers, not words.  How do we translate our headlines into something a machine can understand?

The first step is to create something called a bag of words matrix.  A bag of word matrix gives us a numerical representation of which words are in which headlines.

In order to construct a bag of words matrix, we find the unique words across the whole set of headlines.  Then, we setup a matrix where each row is a headline, and each column is one of the unique words.  Then, we fill in each cell with the number of times that word occured in that headline.

In [32]:
## Initial

import pandas
submissions = pandas.read_csv("hn_stories.csv")
submissions.columns = ["submission_time", "upvotes", "url", "headline"]
submissions = submissions.dropna()

## Display

from collections import Counter

headlines = [
    "PretzelBros, airbnb for people who like pretzels, raises $2 million",
    "Top 10 reasons why Go is better than whatever language you use.",
    "Why working at apple stole my soul (I still love it though)",
    "80 things I think you should do immediately if you use python.",
    "Show HN: carjack.me -- Uber meets GTA"
]

# Find all the unique words in the headlines.
unique_words = list(set(" ".join(headlines).split(" ")))
def make_matrix(headlines, vocab):
    matrix = []
    for headline in headlines:
        # Count each word in the headline, and make a dictionary.
        counter = Counter(headline)
        # Turn the dictionary into a matrix row using the vocab.
        row = [counter.get(w, 0) for w in vocab]
        matrix.append(row)
    df = pandas.DataFrame(matrix)
    df.columns = unique_words
    return df

print(make_matrix(headlines, unique_words))

   like  raises  Why  80  if  you  python.  use.  apple  Uber ...   than  \
0     0       0    0   0   0    0        0     0      0     0 ...      0   
1     0       0    0   0   0    0        0     0      0     0 ...      0   
2     0       0    0   0   0    0        0     0      0     0 ...      0   
3     0       0    0   0   0    0        0     0      0     0 ...      0   
4     0       0    0   0   0    0        0     0      0     0 ...      0   

   million  soul  think  language  people  GTA  my  who  HN:  
0        0     0      0         0       0    0   0    0    0  
1        0     0      0         0       0    0   0    0    0  
2        0     0      0         0       0    0   0    0    0  
3        0     0      0         0       0    0   0    0    0  
4        0     0      0         0       0    0   0    0    0  

[5 rows x 51 columns]


<!--- type="code" --->

# Removing punctuation

The matrix we just made is very sparse -- that means that a lot of the values are zero.  To some extent, this is unavoidable, because the headlines are very different and not very long, but we can take some steps to fix it.  Right now `Why` and `why`, and `use` and `use.` are treated as different, only though they aren't.

We can help the parser recognize that these are in fact the same by lowercasing and removing punctuation.

In [33]:
## Display

import re

# Lowercase, then replace any non-letter, space, or digit character in the headlines.
new_headlines = [re.sub(r'[^\w\s\d]','',h.lower()) for h in headlines]
# Replace sequences of whitespace with a space character.
new_headlines = [re.sub("\s+", " ", h) for h in new_headlines]

unique_words = list(set(" ".join(new_headlines).split(" ")))
# We've reduced the number of columns in the matrix a bit.
print(make_matrix(new_headlines, unique_words))

   like  raises  pretzels  80  if  you  apple  airbnb  for  immediately ...   \
0     0       0         0   0   0    0      0       0    0            0 ...    
1     0       0         0   0   0    0      0       0    0            0 ...    
2     0       0         0   0   0    0      0       0    0            0 ...    
3     0       0         0   0   0    0      0       0    0            0 ...    
4     0       0         0   0   0    0      0       0    0            0 ...    

   soul  think  uber  language  pretzelbros  people  top  gta  my  who  
0     0      0     0         0            0       0    0    0   0    0  
1     0      0     0         0            0       0    0    0   0    0  
2     0      0     0         0            0       0    0    0   0    0  
3     0      0     0         0            0       0    0    0   0    0  
4     0      0     0         0            0       0    0    0   0    0  

[5 rows x 47 columns]


<!--- type="code" --->

# Removing stopwords

Certain words don't give you much discriminative ability between good and bad headlines.  Words such as `the`, `a`, and `also` occur commonly enough in all contexts that they don't really tell us much about whether something is good or not.

By removing these, we can reduce the size of the matrix, and make training an algorithm faster.

In [34]:
## Initial

import re

## Display

# Read in and split the stopwords file.
with open("stop_words.txt", 'r') as f:
    stopwords = f.read().split("\n")

# Do the same punctuation replacement that we did for the headlines, 
# so we're comparing the right things.
stopwords = [re.sub(r'[^\w\s\d]','',s.lower()) for s in stopwords]

unique_words = list(set(" ".join(new_headlines).split(" ")))
# Remove stopwords from the vocabulary.
unique_words = [w for w in unique_words if w not in stopwords]

# We're down to 34 columns, which is way better!
print(make_matrix(new_headlines, unique_words))

   like  raises  pretzels  80  apple  airbnb  immediately  show  reasons  \
0     0       0         0   0      0       0            0     0        0   
1     0       0         0   0      0       0            0     0        0   
2     0       0         0   0      0       0            0     0        0   
3     0       0         0   0      0       0            0     0        0   
4     0       0         0   0      0       0            0     0        0   

   better ...   things  million  soul  think  uber  language  pretzelbros  \
0       0 ...        0        0     0      0     0         0            0   
1       0 ...        0        0     0      0     0         0            0   
2       0 ...        0        0     0      0     0         0            0   
3       0 ...        0        0     0      0     0         0            0   
4       0 ...        0        0     0      0     0         0            0   

   people  top  gta  
0       0    0    0  
1       0    0    0  
2       0    0

<!--- type="code" --->

# Generating a matrix for all the headlines

Now, we want to make a bag of words matrix for the whole set of headlines.  We don't want to have to code everything out manually every time, so we'll use a class from [scikit-learn](http://scikit-learn.org/stable/) to do it automatically.

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

# Construct a bag of words matrix.
# This will lowercase everything, and ignore all punctuation by default.
# It will also remove stop words.
vectorizer = CountVectorizer(lowercase=True, stop_words="english")

matrix = vectorizer.fit_transform(headlines)
# We created our bag of words matrix with far fewer commands.
print(matrix.todense())

# Let's apply it to all the headlines for all the submissions.
# We'll also add the url to the end of the headline so we can take it into account.
submissions['full_test'] = submissions["headline"] + " " + submissions["url"]
full_matrix = vectorizer.fit_transform(submissions["headline"])
print(full_matrix.shape)

[[0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]
(93712, 50571)


<!--- type="code" --->

# Reducing dimensionality

We've constructed a matrix, but it now has `50571` unique words, or columns.  This will take a very long time to make predictions with.  We want to speed it up, so we'll need to cut down on this number somehow.

One way to do this is to pick a subset of the columns that are the most informative -- that is, differentiate between good and bad headlines the best.  A good way to figure out the most informative columns is to use something called a [chi-squared](http://en.wikipedia.org/wiki/Chi-squared_test) test.  

A chi-squared test finds the words that discriminate the most between highly upvoted posts and posts that weren't upvoted.  This can be words that occur a lot in highly upvoted posts, and not at all in posts without upvotes, or words that occur a lot in posts that aren't upvoted, but don't occur in posts that are upvoted.

A chi-squared test only works on binary values, so we'll make our upvotes column binary by setting anything with more upvotes than average to `1` and anything with less upvotes than average to `0`.

One downside of this is that we are using knowledge from the dataset to select features, and thus introducing some overfitting.  We could get around the overfitting in the "real world" by using a subset of the data for feature selection, and using a different subset for training the algorithm.  We'll make things a bit simpler for now and skip that step.

In [36]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Convert the upvotes variable to binary so it works with a chi squared test.
col = submissions["upvotes"].copy(deep=True)
col_mean = col.mean()
col[col < col_mean] = 0
col[(col > 0) & (col > col_mean)] = 1

# Find the 1000 most informative columns
selector = SelectKBest(chi2, k=1000)
selector.fit(full_matrix, col)
top_words = selector.get_support().nonzero()

# Pick only the most informative columns in the data.
chi_matrix = full_matrix[:,top_words[0]]

<!--- type="code" --->

# Adding meta features

If we ignore the "meta" features of the text, we're missing out on a lot of good features.  These features are things like length, amount of punctuation, average word length, and other sentence specific features.

Adding these in can greatly increase prediction accuracy.

To add them in, we'll loop over our headlines, and apply a function to each one.  Some functions will count the length of the headline in characters, and others will do more advanced things, like counting the number of digits.

In [37]:
## Initial

import numpy
import re

## Display

# Our list of functions to apply.
transform_functions = [
    lambda x: len(x),
    lambda x: x.count(" "),
    lambda x: x.count("."),
    lambda x: x.count("!"),
    lambda x: x.count("?"),
    lambda x: len(x) / (x.count(" ") + 1),
    lambda x: x.count(" ") / (x.count(".") + 1),
    lambda x: len(re.findall("\d", x)),
    lambda x: len(re.findall("[A-Z]", x)),
]

# Apply each function and put the results into a list.
columns = []
for func in transform_functions:
    columns.append(submissions["headline"].apply(func))
# Convert the meta features to a numpy array.
meta = numpy.asarray(columns).T

# Add the meta features to the existing text features.
features = numpy.hstack([meta, chi_matrix.todense()])

<!--- type="code" --->

# Adding in more features

There are more features we can work with than just text features.  We have two columns, `id`, and `submission_time`, that add additional information.  Often when doing NLP work, you'll be able to add informative outside features that can make your predictions much better.

In [38]:
columns = []

# Convert the submission dates column to datetime.
submission_dates = pandas.to_datetime(submissions["submission_time"])

# Transform functions for the datetime column.
transform_functions = [
    lambda x: x.year,
    lambda x: x.month,
    lambda x: x.day,
    lambda x: x.hour,
    lambda x: x.minute,
]

# Apply all functions to the datetime column.
for func in transform_functions:
    columns.append(submission_dates.apply(func))

# Convert the meta features to a numpy array.
non_nlp = numpy.asarray(columns).T

# Concatenate the features together.
full_features = numpy.hstack([non_nlp, features])

<!--- type="code" --->

# Making predictions

Now that we can translate words to numbers, we can make predictions using an algorithm.  We'll randomly pick `75000` headlines as a training set, and then evaluate the performance of the algorithm on the test set of `25000` headlines.

For the algorithm, we'll use [ridge regression](http://en.wikipedia.org/wiki/Tikhonov_regularization).  As compared to ordinary linear regression, ridge regression introduces a penalty on the coefficients, which prevents them from becoming too large.  This can help it work with large numbers of predictions (columns), like we have.

In [39]:
from sklearn.linear_model import Ridge
import numpy
import random

train_rows = 75000
random.seed(1)

# Shuffle the indices for the matrix randomly.
indices = list(range(features.shape[0]))
random.shuffle(indices)

# Create train and test sets.
train = full_features[indices[:train_rows], :]
test = full_features[indices[train_rows:], :]
train_upvotes = submissions["upvotes"].iloc[indices[:train_rows]]
test_upvotes = submissions["upvotes"].iloc[indices[train_rows:]]
train = numpy.nan_to_num(train)

# Run the regression and generate predictions for the test set.
reg = Ridge(alpha=10)
reg.fit(train, train_upvotes)
predictions = reg.predict(test)

<!--- type="code" --->

# Evaluating error

We now have predictions, but how do we determine how good they are?  One good way is to calculate the error between the predictions on the test set and the actual upvote counts on the test set.

We'll also want a baseline to compare this to to see if the results are good.  We can do this by using a simple method to make "predictions" for the test set, and comparing the performance of our baseline to that method.  One very simple baseline is an average baseline -- just take the average number of upvotes, and use that as a prediction.

We'll use [mean absolute error](http://en.wikipedia.org/wiki/Mean_absolute_error) as an error metric.  It's very simple -- just subtract the actual value from the prediction, take the absolute value, then add all the differences together and divide by the number of differences to get an average.

In [40]:
# We're going to use mean absolute error as an error metric.
# Our error is about 15.2 upvotes, which means that, on average, 
# our prediction is 15.2 upvotes away from the actual number of upvotes.
print(sum(abs(predictions - test_upvotes)) / len(predictions))

# As a baseline, we'll use the average number of upvotes
# across all submissions.
# The error here is 15.9 -- our estimate is better, but not hugely so.
# There either isn't a ton of predictive value encoded in the 
# data we have, or we aren't extracting it well.
average_upvotes = sum(test_upvotes)/len(test_upvotes)
print(sum(abs(average_upvotes - test_upvotes)) / len(predictions))


15.1950403551
15.9035832815


<!--- type="markdown" --->

# Improvements

This method worked reasonably but not stunningly well on this dataset, and the data had some predictive value.

We could improve this by using a different predictive algorithm, like random forest.  We could also use ngrams, such as bigrams and trigrams, when we are generating our bag of words matrix.  We could also take other data into account, like the user who submitted the article.

Ultimately, all of these methods will take much longer to run than what we have so far, but will likely reduce error a bit.